# IFSP - Câmpus Campinas

## Análise sobre a previsibilidade das decisões de admissibilidade no Superior Tribunal de Justiça usando inteligência artificial

#### Aluno: Guilherme Cioldin Dainese

### Parte 3 - Processamento dos textos

### 3.1 Carregamento dos dados

In [112]:
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
####
import pandas as pd
import numpy as np
import re
import nltk
import neattext.functions as ntx
import spacy
import neattext as nt
import re
import unicodedata
from tqdm import tqdm
from nltk.corpus import stopwords
tqdm.pandas()  # Inicializa a integração do tqdm com pandas

In [68]:
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_feather('./dataframes/texto_raw.feather')
df

,seqDocumento,numeroRegistro,ministro,teor,descricaoMonocratica,assuntos,y,texto
0,178153183,202300036881,JOÃO OTÁVIO DE NORONHA,Negando,Conhecido o recurso e não-provido,9607,0,"[Rua Pedro Ferreira, nº 155 - 7º Andar - Sala ..."
1,181376805,202202985103,NANCY ANDRIGHI,Negando,Agravo conhecido para não conhecer do Recurso ...,9607,0,[ \nPJ 571059\nEXCELENTÍSSIMO SENHOR DOUTOR DE...
2,196160898,202301575526,MARIA THEREZA DE ASSIS MOURA,Não Conhecendo,Não conhecido o recurso,9607,0,[ \nBANDEIRA & BARROS\nAdvocacia e Consultoria...
3,220390496,202302743171,MARIA THEREZA DE ASSIS MOURA,Não Conhecendo,Não conhecido o recurso,9607,0,[Num. 161104655 - Pág. 1\nAssinado eletronicam...
4,216103802,202302956038,MARIA THEREZA DE ASSIS MOURA,Não Conhecendo,Não conhecido o recurso,9607,0,[ \n \n \nEXCELENTÍSSIMO SENHOR DESEMBARGADOR ...
...,...,...,...,...,...,...,...,...
1495,192080215,202300721459,JOÃO OTÁVIO DE NORONHA,Concedendo,Conhecido o recurso e provido,"12401,7779,7780,9607",1,[ \nPágina 1 de 17 \n \n \n \nEXMO. SR. DR. DE...
1496,204842588,202203536482,JOÃO OTÁVIO DE NORONHA,Concedendo,Conhecido o recurso e provido em parte,9607,1,"[ \n \nAv. T 05, Qd. 04, Lote 17, Sala Térrea,..."
1497,179332552,202203993058,MARCO BUZZI,Concedendo,Conhecido o recurso e provido,9607,1,[MARINGÁ/PR \nEndereço: Rua João Paulino Vieir...
1498,190594784,202301534845,NANCY ANDRIGHI,Concedendo,Conhecido o recurso e provido,"11806,9607",1,[ \n \nEXCELENTÍSSIMO SENHOR DOUTOR DESEMBARGA...


In [3]:
#Exemplo de texto extraído de uma página
df.iloc[5]['texto'][5]

' \nJUDICIÁRIA. \nPESSOA \nJURÍDICA. \nPOSSIBILIDADE. \nSÚMULA \n481/STJ. \nDEFERIMENTO. 1. De acordo com a norma \nprevista no art. 1.022 do CPC/2015, são \ncabíveis embargos de declaração nas hipóteses \nde obscuridade, contradição ou omissão da \ndecisão recorrida. 2. No caso, verificada a \nomissão no acórdão embargado, quanto ao \npedido de assistência judiciária gratuita, cabível \no acolhimento dos embargos para apreciação \ndo pleito. 3. Conforme a Súmula 481/STJ, "Faz \njus ao benefício da justiça gratuita a pessoa \njurídica com ou sem fins lucrativos que \ndemonstrar sua impossibilidade de arcar \ncom os encargos processuais. 4. Hipótese \nem que ficou evidenciada a situação de \nhipossuficiência \nfinanceira \nda \npessoa \njurídica embargante, cabendo, por isso, o \ndeferimento do benefício da assistência \njudiciária gratuita em seu favor, conforme \nprevisto no art. 98 do CPC/2015, sem prejuízo \nda ressalva contida no § 3º desse mesmo \ndispositivo. 5. Vigora no Superio

In [4]:
df_backup = df.copy()

### 3.2 Pré-processamento do texto

In [5]:
#Função para aplicar Regex em cada página da lista
def substituir_pagina(lista_paginas, regex_padrão, substituição):
    """
    Aplica uma substituição usando regex em cada página de uma lista.
    
    Parâmetros:
    lista_paginas (list): Lista contendo o texto de cada página.
    regex_padrão (str): A expressão regular a ser usada para encontrar o texto a ser substituído.
    substituição (str): O texto que substituirá as ocorrências encontradas pelo regex.
    
    Retorna:
    list: Lista com as páginas atualizadas após as substituições.
    """
    return [pd.Series(pagina).str.replace(regex_padrão, substituição, regex=True).values[0]
            for pagina in lista_paginas]

In [6]:
#Função para aplicar a função anterior em todo o dataframe
def aplicar_regex(regex_padrão, substituição, dataframe=df):
    """
    Aplica uma substituição usando regex a todas as páginas na coluna 'texto' de um DataFrame.
    
    Parâmetros:
    dataframe (pd.DataFrame): O DataFrame contendo a coluna 'texto' com listas de páginas.
    regex_padrão (str): A expressão regular a ser usada para encontrar o texto a ser substituído.
    substituição (str): O texto que substituirá as ocorrências encontradas pelo regex.
    
    Retorna:
    pd.DataFrame: O DataFrame atualizado com as substituições aplicadas.
    """
    # Aplica a função de substituição à coluna 'texto'
    dataframe['texto'] = dataframe['texto'].apply(substituir_pagina, args=(regex_padrão, substituição))
    return dataframe

In [7]:
#Lista de substituições
#Essa lista contém as palavras ou expressões que serão removidas 
lista_remover1 =[
r'-\n', #Remove as quebras de linha com hífen, ou seja, palavras que quebraram a linha e queremos que sejam juntadas novamente
r'Documento recebido eletronicamente da origem', #Expressão constante em todas as petições baixadas
r'\(e-STJ Fl\.\d+\)' #Numeração presente em todas as páginas dos arquivos
]

In [8]:
#Remove a lista de expressões acima das páginas do dataframe
for i in lista_remover1:
    df = aplicar_regex(i, '')

In [9]:
#Remove quebras de linha
df = aplicar_regex(r'\n', ' ')
#Remove mais de um espaço em branco
df = aplicar_regex(r'\s+', ' ')

In [31]:
# Comando para remover diacríticos do dataframe
df['texto'] = df['texto'].apply(lambda paginas: 
    [''.join(c for c in unicodedata.normalize('NFD', pagina) if unicodedata.category(c) != 'Mn') for pagina in paginas])

In [14]:
#Comando para covnerter todas as palavras em minúsculas
df['texto'] = df['texto'].apply(lambda paginas: [pagina.lower() for pagina in paginas])

In [22]:
#Comando para remover hyperlinks
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'http[s]?://\S+|www\.\S+', '', pagina) for pagina in paginas])

In [29]:
# Comando para remover números no formato CNJ 0000000-00.0000.0.00.0000
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'\b\d{7}-\d{2}\.\d{4}\.\d{1}\.\d{2}\.\d{4}\b', '', pagina) for pagina in paginas])

In [34]:
#Remove mais de um espaço em branco
df = aplicar_regex(r'\s+', ' ')

In [38]:
# Comando para remover a assinatura das petições encaminhadas do TJSP e outros tribunais que usam o SAJ.
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'para conferir o original, acesse o site informe o processo e codigo \w{8}\.', '', pagina, flags=re.IGNORECASE) for pagina in paginas])
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'este documento e copia do original, assinado digitalmente por .*?, protocolado em \d{2}/\d{2}/\d{4} as \d{2}:\d{2} , sob o numero [A-Za-z]{4}\d{11}\.', '', pagina, flags=re.IGNORECASE) for pagina in paginas])

In [43]:
#Remove a expressão 'fls. ... ' com o número das folhas do processo.
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'fls\. \d+', '', pagina) for pagina in paginas])

In [47]:
#Remove a assinatura padrão do PJe
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'assinado eletronicamente por: .*? - \d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}', '', pagina, flags=re.IGNORECASE) for pagina in paginas])

In [51]:
#Remove a assinatura do projudi/pr
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'documento assinado digitalmente, conforme mp nº 2.200-2/2001, lei nº 11.419/2006, resolucao do projudi, do tjpr/oe', '', pagina, flags=re.IGNORECASE) for pagina in paginas])

In [56]:
#Remove a pontuação dos textos
pontuacao = r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¹²³ªº°®©øØ'
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(f'[{re.escape(pontuacao)}]', '', pagina) for pagina in paginas])

In [61]:
#Remove o texto 'página x de x'
df['texto'] = df['texto'].apply(lambda paginas: 
    [re.sub(r'pagina \d{1,3} de \d{1,3}', '', pagina, flags=re.IGNORECASE) for pagina in paginas])

In [69]:
# Comando para remover palavras de uma letra só e stopwords
df['texto'] = df['texto'].apply(lambda paginas: 
    [' '.join([palavra for palavra in re.findall(r'\b\w+\b', pagina) if len(palavra) > 1 and palavra.lower() not in stop_words]) for pagina in paginas])

-------

In [74]:
#cria a coluna palavra comuns
df['palavrasComuns'] = None

In [75]:
#Função para encontrar palavras comuns em todas as páginas de um recurso
def encontrar_palavras_comuns(lista_textos):
    # Verifica se a lista está vazia ou se não contém textos
    if lista_textos is None or len(lista_textos) == 0:
        return set()  # Retorna um conjunto vazio se não houver páginas
    
    # Inicializa o conjunto com as palavras da primeira página, considerando palavras não vazias
    palavras_comuns = set(lista_textos[0].split())
    
    # Itera pelas páginas restantes e realiza a interseção
    for texto in lista_textos[1:]:
        palavras_comuns.intersection_update(texto.split())
    
    # Retorna as palavras comuns, ou None se o conjunto estiver vazio
    return palavras_comuns if palavras_comuns else None

In [76]:
# Supondo que o DataFrame se chame df e a coluna com as listas de textos seja 'texto_por_pagina'
df['palavrasComuns'] = df['texto'].apply(encontrar_palavras_comuns)

In [85]:
# Função para remover palavras constantes do campo 'palavrasComuns'
def remover_palavras_comuns(row):
    palavras_comuns = row['palavrasComuns']
    
    # Verifica se palavrasComuns não é None
    if palavras_comuns is None:
        return row['texto']  # Retorna o texto original se palavrasComuns for None

    texto_filtrado = []
    
    for pagina in row['texto']:
        if pagina is not None:  # Verifica se a página não é None
            # Remove as palavras comuns usando regex
            pagina_filtrada = ' '.join([palavra for palavra in re.findall(r'\b\w+\b', pagina) if palavra.lower() not in palavras_comuns])
            texto_filtrado.append(pagina_filtrada)
        else:
            texto_filtrado.append(None)  # Adiciona None se a página for None
    
    return texto_filtrado

In [86]:
# Aplicar a função no DataFrame
df['texto'] = df.apply(remover_palavras_comuns, axis=1)

----
Texto está processado, hora de colocar em uma única coluna

In [89]:
df['textoProcessado'] = None

In [91]:
# Função para juntar o texto de todas as páginas
def juntar_texto(paginas):
    if paginas is None:  # Verifica se as páginas são None
        return None
    return ' '.join(paginas)  # Junta todas as páginas em uma única string

In [92]:
# Criar a nova coluna 'textoProcessado'
df['textoProcessado'] = df['texto'].apply(juntar_texto)

Hora de dividir entre treino e teste e testar os modelos

In [94]:
# Define X and y
X = df['textoProcessado']
y = df['y']

In [126]:
#Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [127]:
# Define a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())])

In [128]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [129]:
# Predictions on the test set
predictions = pipeline.predict(X_test)

In [130]:
# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.75      0.73       146
           1       0.75      0.72      0.74       154

    accuracy                           0.73       300
   macro avg       0.73      0.73      0.73       300
weighted avg       0.73      0.73      0.73       300



In [134]:
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       146
           1       0.79      0.79      0.79       154

    accuracy                           0.79       300
   macro avg       0.79      0.79      0.79       300
weighted avg       0.79      0.79      0.79       300



In [136]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.78      0.74       146
           1       0.77      0.69      0.73       154

    accuracy                           0.74       300
   macro avg       0.74      0.74      0.74       300
weighted avg       0.74      0.74      0.74       300



In [139]:
from xgboost import XGBClassifier

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, predictions))

/venvs/TCC/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:22:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.74      0.75       146
           1       0.76      0.78      0.77       154

    accuracy                           0.76       300
   macro avg       0.76      0.76      0.76       300
weighted avg       0.76      0.76      0.76       300



In [140]:
from sklearn.naive_bayes import BernoulliNB

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', BernoulliNB())
])
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.79      0.72       146
           1       0.76      0.61      0.68       154

    accuracy                           0.70       300
   macro avg       0.71      0.70      0.70       300
weighted avg       0.71      0.70      0.70       300



In [131]:

# Acessar o TfidfVectorizer do pipeline
tfidf_vectorizer = pipeline.named_steps['tfidf']
# Acessar o classificador
classifier = pipeline.named_steps['clf']

# Obter os nomes das características (palavras)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Obter as probabilidades logarítmicas
log_probs = classifier.feature_log_prob_

# Para cada classe, obter as palavras mais significativas
for class_index in range(log_probs.shape[0]):
    # Criar um DataFrame com as palavras e suas probabilidades logarítmicas
    word_probs = pd.DataFrame({
        'word': feature_names,
        'log_prob': log_probs[class_index]
    })

    # Calcular a importância das palavras
    word_probs['importance'] = word_probs['log_prob'] - log_probs.mean(axis=1)[class_index]

    # Ordenar as palavras pela importância
    significant_words = word_probs.sort_values(by='importance', ascending=False)

    # Exibir as 10 palavras mais significativas para a classe
    print(f'Palavras mais significativas para a classe {class_index}:')
    print(significant_words.head(100))

Palavras mais significativas para a classe 0:
             word  log_prob  importance
59242         nao -7.007749    4.327962
68702     recurso -7.344974    3.990737
28881         art -7.698907    3.636803
76641    tribunal -7.800974    3.534736
44438    especial -7.910946    3.424764
...           ...       ...         ...
65753  processual -9.139172    2.196538
46671        fato -9.144658    2.191053
78410    violacao -9.150354    2.185356
30801   beneficio -9.152061    2.183649
64026        pois -9.153488    2.182223

[100 rows x 3 columns]
Palavras mais significativas para a classe 1:
             word  log_prob  importance
59242         nao -7.015265    4.321478
68702     recurso -7.452426    3.884317
28881         art -7.562240    3.774503
54842       juros -7.590504    3.746239
75167        taxa -7.694236    3.642508
...           ...       ...         ...
66418    proveito -9.052249    2.284495
65127  prescricao -9.063971    2.272773
75125     tarifas -9.067854    2.268889
7266

In [133]:
# Obter os nomes das características (palavras)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Obter as probabilidades logarítmicas
log_probs = classifier.feature_log_prob_

# Configurar o Pandas para mostrar todas as linhas
pd.set_option('display.max_rows', None)  # None para mostrar todas as linhas
pd.set_option('display.max_columns', None)  # None para mostrar todas as colunas

# Para cada classe, obter as palavras mais significativas
for class_index in range(log_probs.shape[0]):
    # Criar um DataFrame com as palavras e suas probabilidades logarítmicas
    word_probs = pd.DataFrame({
        'word': feature_names,
        'log_prob': log_probs[class_index]
    })

    # Calcular a importância das palavras
    word_probs['importance'] = word_probs['log_prob'] - log_probs.mean(axis=1)[class_index]

    # Ordenar as palavras pela importância
    significant_words = word_probs.sort_values(by='importance', ascending=False)

    # Exibir todas as palavras significativas para a classe
    print(f'Palavras mais significativas para a classe {class_index}:')
    print(significant_words.head(100))


Palavras mais significativas para a classe 0:
                    word  log_prob  importance
59242                nao -7.007749    4.327962
68702            recurso -7.344974    3.990737
28881                art -7.698907    3.636803
76641           tribunal -7.800974    3.534736
44438           especial -7.910946    3.424764
54894            justica -7.931669    3.404041
25155            acordao -7.952622    3.383088
65714           processo -8.115502    3.220208
68552         recorrente -8.115974    3.219736
38184            decisao -8.122858    3.212852
65158           presente -8.166324    3.169386
33332              civil -8.212248    3.123462
55539                lei -8.238977    3.096733
29013             artigo -8.248563    3.087148
54842              juros -8.269853    3.065857
30012              autos -8.297427    3.038283
46824            federal -8.311150    3.024560
62561              parte -8.321246    3.014464
40947            direito -8.323913    3.011798
36282         